In [1]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#########################################
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go


READING DATA, INITIALIZATION

In [2]:
whole_data = pd.read_csv('C:/Users/halil/Desktop/forecast_co2/datasets/ethylene_CO.txt', delimiter='\s+',names=[
   "Time",
   "CO2_con",
   "Ethy_con",
   "sensor1",
   "sensor2",
   "sensor3",
   "sensor4",
   "sensor5",
   "sensor6",
   "sensor7",
   "sensor8",
   "sensor9",
   "sensor10",
   "sensor11",
   "sensor12",
   "sensor13",
   "sensor14",
   "sensor15",
   "sensor16",])

#normalization in range (-1,1)

In [3]:

def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

function that create train test data and labels from raw dataset

In [4]:

def make_data(dataframe):
    train = dataframe[:int(len(dataframe)*0.8)]
    test = dataframe[int(len(dataframe)*0.8):]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('CO2_con')
    y_test = x_test.pop('CO2_con')

    return x_train, x_test, y_train, y_test 

machine learning model function, to get model errors 

In [5]:
def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_linear_reg(x_train,x_test,y_train,y_test):

    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

In [6]:
df= whole_data.copy()

#new_data = df.drop(["Time"],axis=1)
#co2_data = new_data.drop(["Ethy_con"],axis=1)
#eth_data = new_data.drop(["CO2_con"],axis=1)

In [ ]:
orginized_time = new_data.copy()
orginized_time["Time"] = pd.to_datetime(orginized_time["Time"], unit="ms",origin="2022")
orginized_time['Time'] = orginized_time.Time.dt.round("10U")
#time_series = df[["Time","sensor1"]]
#df = df.set_index('Time')